In [1]:
import numpy as np
import jax.numpy as jnp
import flax.linen as nn
import matplotlib.pyplot as plt
import os
from tifffile import imread, imsave
import torch
import torch.utils.data as dt

In [2]:
os.chdir('/mnt/c/Users/ankit/Desktop/Msc AIML/msc-project/GAP-JAX')
from gap_jax.BinomDataset_JAX import BinomDataset
from  gap_jax.GAP_JAX_Trainer import Trainer
from gap_jax.GAP_UNET_ResBlock_JAX import UN

In [3]:
data = imread('/mnt/c/Users/ankit/Desktop/Msc AIML/Data/convallaria/trainingDataGT.tif')

In [4]:
print(data.shape)

(1536, 512, 512)


In [5]:
def psnrToString(inp):
    if inp < 0:
        return 'm'+str(-inp)
    else:
        return str(inp)

minpsnr = -40
maxpsnr = -5

name = psnrToString(minpsnr)+"to"+psnrToString(maxpsnr)+"-256x256-jax-v1"
CHECKPOINT_PATH = '/mnt/c/Users/ankit/Desktop/Msc AIML/Data/checkpoints/models/'
CHECKPOINT_PATH, name

('/mnt/c/Users/ankit/Desktop/Msc AIML/Data/checkpoints/models/',
 'm40tom5-256x256-jax-v1')

In [6]:
maxepochs = 20 # train longer for better results

# Create pytorch dataset
dataset = BinomDataset(data = data[:round(data.shape[0]*0.9)], windowSize = 256, minPSNR = minpsnr, maxPSNR = maxpsnr)
val_dataset = BinomDataset(data = data[round(data.shape[0]*0.9):], windowSize = 256, minPSNR = minpsnr, maxPSNR = maxpsnr)

# Create pytorch dataloader for training and validation sets
train_loader = dt.DataLoader(dataset, batch_size=8, shuffle=True, drop_last=True, pin_memory=False, num_workers=12, collate_fn= lambda batch: np.array(batch)) ## Changing the batch size from 32 to 16 to fit inside the gpu
val_loader = dt.DataLoader(val_dataset, batch_size=8, shuffle=False, drop_last=False,  pin_memory=False, num_workers=12, collate_fn= lambda batch: np.array(batch))

img =  next(iter(val_loader))

print(img.shape)

(8, 256, 256, 2)


In [7]:
model = UN(channels = 1, levels=10, depth=3, start_filts=28, up_mode = 'transpose')

In [8]:
trainer = Trainer(root_dir= os.path.join(CHECKPOINT_PATH, name),
                  gradient_clip_val=0.5,
                  epochs= maxepochs,
                  dataloader= train_loader,
                  model = model)

GPU detected with 1 Device(s).


In [9]:
trainer.train_model(train_loader= train_loader,
                    val_loader= val_loader)

  0%|          | 0/20 [00:00<?, ?it/s]

/home/ankith/anaconda3/envs/rsenv/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Training:   0%|          | 0/172 [00:00<?, ?it/s]

TypeError: reduce_on_plateau.<locals>.update_fn() missing 1 required keyword-only argument: 'value'